In [68]:
import numpy as np
import matplotlib.pyplot as plt

from create_prepared_data_tsfresh import *
from compression import calculateCompRatio, compress_dataset
from utils.personal_utils import *

import os
import pywt

import matplotlib.pyplot as plt
import time

In [69]:
all_ds = {
    'AppliancesEnergy':   '/home/sim/Desktop/TS Extrinsic Regression/data/AppliancesEnergy_TEST.ts',
    'NewsTitleSentiment': '/home/sim/Desktop/TS Extrinsic Regression/data/NewsTitleSentiment_TEST.ts',
    'BenzeneConcentration':'/home/sim/Desktop/TS Extrinsic Regression/data/BenzeneConcentration_TEST.ts',
    'BeijingPM25Quality': '/home/sim/Desktop/TS Extrinsic Regression/data/BeijingPM25Quality_TEST.ts',

    'IEEEPPG':            '/home/sim/Desktop/TS Extrinsic Regression/data/IEEEPPG_TEST.ts',
    'FloodModeling1':     '/home/sim/Desktop/TS Extrinsic Regression/data/FloodModeling1_TEST.ts',
    'HouseholdPowerConsumption1': '/home/sim/Desktop/TS Extrinsic Regression/data/HouseholdPowerConsumption1_TEST.ts',
    'Covid3Month':              '/home/sim/Desktop/TS Extrinsic Regression/data/Covid3Month_TEST.ts'
}

Steps:

Keep in mind: 
- Ignore 0.99 for now, to have more overlapping features.
- Selected features come from train_set and are then same for test_set
- Correlation Analysis on TRAIN Set: Same as for TEST, just to get y value let the pkl. model run and get the target value!
- When redoing runs, consider keeping more features! (Maybe no AutoMl featuere selectin! Or analyse both!)
- I assume the rows in the logged prediction and loaded df are in the same order. -> but should be! -> if not error wouldn't make sense


Found out: 
- Code is alright
- Removing None or 0.99 or anything else gives bach a few features for each ds. But not really much.
-> Can consider removing 0.99 from comparing. Too much compression different features.
- The dim are already in the .txt file not in order!
- Both functions seem to work. Simple test.
- Also no overlapping features for comp_algos if ignoring Appliances


- Can still do Correlation Analysis for some Datasets and Include in Thesis.


In [70]:
# Get the overlapping features for each (Dataset, dropout) we go over all tq
dropout_values = [None, 0.5, 0.75, 0.85, 0.9, 0.95, 0.97, 0.99]
log_path = '/home/sim/Desktop/TS Extrinsic Regression/All-Logs/' 

ds_names = [
    'AppliancesEnergy',
    #'NewsTitleSentiment',
    'BenzeneConcentration',
    'BeijingPM25Quality',
    'IEEEPPG',
    'FloodModeling1',
    'HouseholdPowerConsumption1',
    'Covid3Month'
]

tqs = ['dct', 'dft', 'dwt']

common_features_dval = {}

# Helper function to find common elements in lists
def common_elements(lists):
    if not lists:
        return []
    common = lists[0]
    for lst in lists[1:]:
        common = [elem for elem in common if elem in lst]
    return common


# Iterate over them to go for each value
for ds_name in ds_names:

    for dval in dropout_values:
        path_to_runs = log_path + f'{ds_name}_Runs/'
        common_features = None
        features_lists = []
        
        for tq in tqs:
            if dval is None:
                path_to_selectedf = path_to_runs + f'NONE_{ds_name}_20min_Flaml_f/selected_features.txt'
            else:
                path_to_selectedf = path_to_runs + f'{dval}_{tq}_{ds_name}_20min_Flaml_f/selected_features.txt'
            
            try:
                with open(path_to_selectedf, 'r') as file:
                    lines = file.read().splitlines()  # Read and store lines in a list
                    features_lists.append(lines)
                        
            except FileNotFoundError:
                print(f"File {path_to_selectedf} not found, skipping.")
                continue
        
        common_features = common_elements(features_lists)
        common_features_dval[(ds_name, dval)] = common_features



In [71]:
# Print overlapping for (Dataset,dval) combinaton
for (ds_name, dval), common_features in common_features_dval.items():
    if common_features is not None:
        print(f"Number of overlapping features for {ds_name}: {dval} {len(common_features)}")
print()



# Print num of overlapping for each dval
overlap_per_dval = {}
for tmp_dval in dropout_values:
        overlap_per_dval[tmp_dval] = common_elements([common_features for (ds_name, dval), common_features in common_features_dval.items() if dval == tmp_dval])
        print(f"Number of overlapping features for whole {tmp_dval} {len(overlap_per_dval[tmp_dval])}")
print()


# Print number of overlapping for each dataset
overlapp_per_dataset = {}
for tmp_dsname in ds_names:

        overlapp_per_dataset[tmp_dsname] = common_elements([common_features for (ds_name, dval), common_features in common_features_dval.items() if ds_name == tmp_dsname])
        print(f"Number of overlapping features for whole {tmp_dsname} {len(overlapp_per_dataset[tmp_dsname])}")

Number of overlapping features for AppliancesEnergy: None 1187
Number of overlapping features for AppliancesEnergy: 0.5 47
Number of overlapping features for AppliancesEnergy: 0.75 39
Number of overlapping features for AppliancesEnergy: 0.85 40
Number of overlapping features for AppliancesEnergy: 0.9 34
Number of overlapping features for AppliancesEnergy: 0.95 28
Number of overlapping features for AppliancesEnergy: 0.97 47
Number of overlapping features for AppliancesEnergy: 0.99 48
Number of overlapping features for BenzeneConcentration: None 2527
Number of overlapping features for BenzeneConcentration: 0.5 1054
Number of overlapping features for BenzeneConcentration: 0.75 1104
Number of overlapping features for BenzeneConcentration: 0.85 1021
Number of overlapping features for BenzeneConcentration: 0.9 821
Number of overlapping features for BenzeneConcentration: 0.95 590
Number of overlapping features for BenzeneConcentration: 0.97 519
Number of overlapping features for BenzeneConcen

In [73]:
# Calculate the correlation, we get a dict looking like: correlations[('AppliancesEnergy', 'dft', 0.5)] -> pd. of correlation
# But now with different features list, regarding the overlapping features per dval!

# We get each dataset and each dval, get then the feature list for that and then save the the features_df for each tq.


dropout_values = [None, 0.5, 0.75, 0.85, 0.9, 0.95, 0.97, 0.99]
log_path = '/home/sim/Desktop/TS Extrinsic Regression/All-Logs/' 

ds_names = [
    'FloodModeling1',
    'HouseholdPowerConsumption1',
    'Covid3Month',
    'AppliancesEnergy',
    #'NewsTitleSentiment',
    'BenzeneConcentration',
    'BeijingPM25Quality',
    'IEEEPPG',
]

tqs = ['dct', 'dft', 'dwt']

prepared_dfs_dval = {}
correlations_dval = {}



for ds_name in ds_names:
    path_to_runs = log_path + f'{ds_name}_Runs/'



    df_raw = pd.read_parquet('/home/sim/Desktop/TS Extrinsic Regression/features_dfs/' + ds_name + f'/NONE_NONE_{ds_name}_features_TEST')
    #df_raw = load_and_prepare_everything(all_ds[ds_name], None, None)[1].drop('target', axis=1)
    
    path_to_pred = path_to_runs + f'NONE_{ds_name}_20min_Flaml_f/predictions.npy'
    prediction_raw = np.load(path_to_pred)


    for dval in dropout_values[1:]:
        ovf_list_dval = common_features_dval[(ds_name, dval)]

        df_raw_ov_dval = df_raw[ovf_list_dval].copy() # Select overlapping features
        df_raw_ov_dval['prediction'] = prediction_raw

        prepared_dfs_dval[(ds_name,dval,None)] = df_raw_ov_dval.copy() # bc different features for each technique



        for tq in tqs:

            features = pd.read_parquet('/home/sim/Desktop/TS Extrinsic Regression/features_dfs/' + ds_name + f'/{dval}_{tq}_{ds_name}_features_TEST')
            #features = load_and_prepare_everything(all_ds[ds_name], tq, dval)[1].drop('target', axis=1)
            
            ovf_dval = features[ovf_list_dval]

            path_to_pred = path_to_runs + f'{dval}_{tq}_{ds_name}_20min_Flaml_f/predictions.npy'
            prediction = np.load(path_to_pred)


            ovf_dval['prediction'] = prediction
            prepared_dfs_dval[(ds_name, tq, dval)] = ovf_dval.copy()


            # Add Subtraction and Correlation
            sub_df = (df_raw_ov_dval - ovf_dval)
            corrs = sub_df.corrwith(sub_df['prediction'])
            correlations_dval[(ds_name,tq,dval)] = corrs
            print('loaded' + ds_name + tq + str(dval))

loadedFloodModeling1dct0.5
loadedFloodModeling1dft0.5
loadedFloodModeling1dwt0.5
loadedFloodModeling1dct0.75
loadedFloodModeling1dft0.75
loadedFloodModeling1dwt0.75
loadedFloodModeling1dct0.85
loadedFloodModeling1dft0.85
loadedFloodModeling1dwt0.85
loadedFloodModeling1dct0.9
loadedFloodModeling1dft0.9
loadedFloodModeling1dwt0.9
loadedFloodModeling1dct0.95
loadedFloodModeling1dft0.95
loadedFloodModeling1dwt0.95
loadedFloodModeling1dct0.97
loadedFloodModeling1dft0.97
loadedFloodModeling1dwt0.97
loadedFloodModeling1dct0.99
loadedFloodModeling1dft0.99
loadedFloodModeling1dwt0.99
loadedHouseholdPowerConsumption1dct0.5
loadedHouseholdPowerConsumption1dft0.5
loadedHouseholdPowerConsumption1dwt0.5
loadedHouseholdPowerConsumption1dct0.75
loadedHouseholdPowerConsumption1dft0.75
loadedHouseholdPowerConsumption1dwt0.75
loadedHouseholdPowerConsumption1dct0.85
loadedHouseholdPowerConsumption1dft0.85
loadedHouseholdPowerConsumption1dwt0.85
loadedHouseholdPowerConsumption1dct0.9
loadedHouseholdPowerCo

In [82]:
# Get the Average Corr and top 5 of them per (ds_name, dval)

# Get Average Correlatin of absolute value over each (ds_name, tq)
# The Entries are Series object, the row name is the featuer name!
corr_ds_name_tq_dval = {}


for ds_name in ds_names:
    

    for dval in dropout_values[1:]:

        # Initalize a df with same columns, rows but zero in each field
        avg_corr = correlations_dval[(ds_name,'dct', dval)].copy()
        avg_corr[:] = 0
        #print(avg_corr)

        for tq in tqs:
            curr_corr_df = correlations_dval[(ds_name,tq, dval)].abs()  # take abs value
            avg_corr = curr_corr_df + avg_corr
        
        avg_corr = avg_corr / len(tqs)
    
        
        corr_ds_name_tq_dval[(ds_name, dval)] = avg_corr.sort_values(ascending=False)


    



In [84]:
# Print for Each Combination the top 5 features
for ds_name in ds_names:
    for dval in dropout_values[1:]:
        print(ds_name + " " + str(dval))
        print(corr_ds_name_tq_dval[(ds_name, dval)].head(6))
        print()

FloodModeling1 0.5
prediction                                                    1.000000
dim_1__ar_coefficient__coeff_0__k_10                          0.298292
dim_1__fft_coefficient__attr_angle__coeff_58                  0.185581
dim_1__augmented_dickey_fuller__attr_usedlag__autolag_AIC     0.146241
dim_1__augmented_dickey_fuller__attr_teststat__autolag_AIC    0.133479
dim_1__fft_coefficient__attr_angle__coeff_53                  0.113192
dtype: float64

FloodModeling1 0.75
prediction                                           1.000000
dim_1__fft_coefficient__attr_angle__coeff_51         0.185052
dim_1__friedrich_coefficients__coeff_0__m_3__r_30    0.138858
dim_1__fft_aggregated__aggtype_skew                  0.131873
dim_1__fourier_entropy__bins_100                     0.124294
dim_1__fft_coefficient__attr_angle__coeff_61         0.124103
dtype: float64

FloodModeling1 0.85
prediction                                      1.000000
dim_1__mean_abs_change                          0.1647

In [ ]:
# Get the overlapping features for each (Dataset, Technique)
dropout_values = [None, 0.5, 0.75, 0.85, 0.9, 0.95, 0.97]
log_path = '/home/sim/Desktop/TS Extrinsic Regression/All-Logs/' 

ds_names = [
    'AppliancesEnergy',
    'NewsTitleSentiment',
    'BenzeneConcentration',
    'BeijingPM25Quality',
    'IEEEPPG',
    'FloodModeling1',
    'HouseholdPowerConsumption1',
    'Covid3Month'
]

tqs = ['dct', 'dft', 'dwt']

common_features_dict = {}

# Helper function to find common elements in lists
def common_elements(lists):
    if not lists:
        return []
    common = lists[0]
    for lst in lists[1:]:
        common = [elem for elem in common if elem in lst]
    return common

# Iterate over them to go for each value
for ds_name in ds_names:


    for tq in tqs:
        path_to_runs = log_path + f'{ds_name}_Runs/'
        common_features = None
        features_lists = []
        
        for dval in dropout_values:
            if dval is None:
                path_to_selectedf = path_to_runs + f'NONE_{ds_name}_20min_Flaml_f/selected_features.txt'
            else:
                path_to_selectedf = path_to_runs + f'{dval}_{tq}_{ds_name}_20min_Flaml_f/selected_features.txt'
            
            try:
                with open(path_to_selectedf, 'r') as file:
                    lines = file.read().splitlines()  # Read and store lines in a list
                    features_lists.append(lines)
                        
            except FileNotFoundError:
                print(f"File {path_to_selectedf} not found, skipping.")
                continue
        
        common_features = common_elements(features_lists)
        common_features_dict[(ds_name, tq)] = common_features





# Print overlapping for (Dataset,Comp_Alg) combinaton
for (ds_name, tq), common_features in common_features_dict.items():
    if common_features is not None:
        print(f"Number of overlapping features for {ds_name}: {tq} {len(common_features)}")
print()



# Print num of overlapping for each tq
overlap_per_tq = {}
for tmp_tq in tqs:
        overlap_per_tq[tmp_tq] = common_elements([common_features for (ds_name, tq), common_features in common_features_dict.items() if tq == tmp_tq])
        print(f"Number of overlapping features for whole {tmp_tq} {len(overlap_per_tq[tmp_tq])}")
print()


# Print number of overlapping for each dataset
overlapp_per_dataset = {}
for tmp_dsname in ds_names:

        overlapp_per_dataset[tmp_dsname] = common_elements([common_features for (ds_name, tq), common_features in common_features_dict.items() if ds_name == tmp_dsname])
        print(f"Number of overlapping features for whole {tmp_dsname} {len(overlapp_per_dataset[tmp_dsname])}")

Number of overlapping features for AppliancesEnergy: dct 4
Number of overlapping features for AppliancesEnergy: dft 1
Number of overlapping features for AppliancesEnergy: dwt 2
Number of overlapping features for NewsTitleSentiment: dct 78
Number of overlapping features for NewsTitleSentiment: dft 63
Number of overlapping features for NewsTitleSentiment: dwt 122
Number of overlapping features for BenzeneConcentration: dct 230
Number of overlapping features for BenzeneConcentration: dft 208
Number of overlapping features for BenzeneConcentration: dwt 235
Number of overlapping features for BeijingPM25Quality: dct 81
Number of overlapping features for BeijingPM25Quality: dft 80
Number of overlapping features for BeijingPM25Quality: dwt 90
Number of overlapping features for IEEEPPG: dct 151
Number of overlapping features for IEEEPPG: dft 117
Number of overlapping features for IEEEPPG: dwt 108
Number of overlapping features for FloodModeling1: dct 29
Number of overlapping features for FloodM

In [ ]:
# Calculate the correlation, we get a dict looking like: correlations[('AppliancesEnergy', 'dft', 0.5)] -> pd. of correlation
dropout_values = [None, 0.5, 0.75, 0.85, 0.9, 0.95, 0.97]
log_path = '/home/sim/Desktop/TS Extrinsic Regression/All-Logs/' 

ds_names = [
    'FloodModeling1',
    'HouseholdPowerConsumption1',
    'Covid3Month',
    'AppliancesEnergy',
    #'NewsTitleSentiment',
    'BenzeneConcentration',
    'BeijingPM25Quality',
    'IEEEPPG',
]


tqs = ['dct', 'dft', 'dwt']

prepared_dfs = {}
correlations = {}

for ds_name in ds_names:
    path_to_runs = log_path + f'{ds_name}_Runs/'



    df_raw = pd.read_parquet('/home/sim/Desktop/TS Extrinsic Regression/features_dfs/' + ds_name + f'/NONE_NONE_{ds_name}_features_TEST')
    #df_raw = load_and_prepare_everything(all_ds[ds_name], None, None)[1].drop('target', axis=1)
    
    path_to_pred = path_to_runs + f'NONE_{ds_name}_20min_Flaml_f/predictions.npy'
    prediction_raw = np.load(path_to_pred)


    for tq in tqs:
        ovf_list = common_features_dict[(ds_name, tq)]

        df_raw_tq = df_raw[ovf_list].copy() # Select overlapping features
        df_raw_tq['prediction'] = prediction_raw

        prepared_dfs[(ds_name,tq,None)] = df_raw_tq.copy() # bc different features for each technique




        for dval in dropout_values[1:]:
            

            features = pd.read_parquet('/home/sim/Desktop/TS Extrinsic Regression/features_dfs/' + ds_name + f'/{dval}_{tq}_{ds_name}_features_TEST')
            #features = load_and_prepare_everything(all_ds[ds_name], tq, dval)[1].drop('target', axis=1)
            
            
            ovf_df = features[ovf_list]
            path_to_pred = path_to_runs + f'{dval}_{tq}_{ds_name}_20min_Flaml_f/predictions.npy'

            try:
                with open(path_to_pred, 'r') as file:
                    prediction = np.load(path_to_pred)
            except FileNotFoundError:
                print(f"File {path_to_pred} not found, skipping.")
                continue

            ovf_df['prediction'] = prediction
            prepared_dfs[(ds_name,tq, dval)] = ovf_df.copy()


            # Add Subtraction and Correlation
            sub_df = (df_raw_tq - ovf_df)
            corrs = sub_df.corrwith(sub_df['prediction'])
            correlations[(ds_name,tq,dval)] = corrs
            print('loaded' + ds_name + tq + str(dval))



loadedFloodModeling1dct0.5
loadedFloodModeling1dct0.75
loadedFloodModeling1dct0.85
loadedFloodModeling1dct0.9
loadedFloodModeling1dct0.95
loadedFloodModeling1dct0.97
loadedFloodModeling1dft0.5
loadedFloodModeling1dft0.75
loadedFloodModeling1dft0.85
loadedFloodModeling1dft0.9
loadedFloodModeling1dft0.95
loadedFloodModeling1dft0.97
loadedFloodModeling1dwt0.5
loadedFloodModeling1dwt0.75
loadedFloodModeling1dwt0.85
loadedFloodModeling1dwt0.9
loadedFloodModeling1dwt0.95
loadedFloodModeling1dwt0.97
loadedHouseholdPowerConsumption1dct0.5
loadedHouseholdPowerConsumption1dct0.75
loadedHouseholdPowerConsumption1dct0.85
loadedHouseholdPowerConsumption1dct0.9
loadedHouseholdPowerConsumption1dct0.95
loadedHouseholdPowerConsumption1dct0.97
loadedHouseholdPowerConsumption1dft0.5
loadedHouseholdPowerConsumption1dft0.75
loadedHouseholdPowerConsumption1dft0.85
loadedHouseholdPowerConsumption1dft0.9
loadedHouseholdPowerConsumption1dft0.95
loadedHouseholdPowerConsumption1dft0.97
loadedHouseholdPowerConsum

In [ ]:
# Get Average Correlatin of absolute value over each (ds_name, tq)
# The Entries are Series object, the row name is the featuer name!
corr_ds_name_tq = {}


for ds_name in ds_names:
    

    for tq in tqs:

        # Initalize a df with same columns, rows but zero in each field
        avg_corr = correlations[(ds_name,tq,0.5)].copy()
        avg_corr[:] = 0
        #print(avg_corr)

        for d_val in dropout_values[1:]:
            curr_corr_df = correlations[(ds_name,tq,d_val)].abs()  # take abs value
            avg_corr = curr_corr_df + avg_corr
        
        avg_corr = avg_corr / len(dropout_values[1:])
    
        
        corr_ds_name_tq[(ds_name, tq)] = avg_corr.sort_values(ascending=False)



corr_ds_name = {}
for ds_name in ds_names:
    corr_ds_name[ds_name] = corr_ds_name_tq[(ds_name, 'dct')][overlapp_per_dataset[ds_name]] 
    corr_ds_name[ds_name] += corr_ds_name_tq[(ds_name, 'dft')][overlapp_per_dataset[ds_name]] 
    corr_ds_name[ds_name] += corr_ds_name_tq[(ds_name, 'dwt')][overlapp_per_dataset[ds_name]] 
    corr_ds_name[ds_name] = corr_ds_name[ds_name].sort_values(ascending=False) / 3
    




In [ ]:
# Get Correlation over each dataset

for ds_name in ds_names:
    print(ds_name )
    print(corr_ds_name[(ds_name)].head(11))
    print()


AppliancesEnergy
dim_6__index_mass_quantile__q_0.6    0.137368
dtype: float64

BenzeneConcentration
dim_6__cwt_coefficients__coeff_0__w_2__widths_(2 5 10 20)           0.050246
dim_5__fft_coefficient__attr_real__coeff_10                         0.047072
dim_3__fft_coefficient__attr_imag__coeff_12                         0.035424
dim_8__fft_coefficient__attr_abs__coeff_18                          0.031933
dim_7__energy_ratio_by_chunks__num_segments_10__segment_focus_0     0.029249
dim_3__fft_coefficient__attr_angle__coeff_20                        0.028338
dim_6__fft_coefficient__attr_angle__coeff_20                        0.028131
dim_8__agg_linear_trend__attr_intercept__chunk_len_50__f_agg_var    0.026458
dim_8__linear_trend__attr_pvalue                                    0.024175
dim_8__index_mass_quantile__q_0.8                                   0.021299
dim_6__fft_coefficient__attr_angle__coeff_10                        0.021238
dtype: float64

BeijingPM25Quality
dim_4__skewness   

In [ ]:
# Print for Each Combination the top 5 features
for ds_name in ds_names:
    for tq in tqs:
        print(ds_name + " " + tq)
        print(corr_ds_name_tq[(ds_name, tq)].head(6))
        print()



AppliancesEnergy dct
prediction                                            1.000000
dim_6__index_mass_quantile__q_0.6                     0.211155
dim_1__agg_autocorrelation__f_agg_mean__maxlag_40     0.187324
dim_23__agg_autocorrelation__f_agg_mean__maxlag_40    0.173494
dim_5__quantile__q_0.6                                0.054032
dtype: float64

AppliancesEnergy dft
prediction                           1.000000
dim_6__index_mass_quantile__q_0.6    0.115208
dtype: float64

AppliancesEnergy dwt
prediction                                           1.000000
dim_1__agg_autocorrelation__f_agg_mean__maxlag_40    0.170318
dim_6__index_mass_quantile__q_0.6                    0.085739
dtype: float64

BenzeneConcentration dct
prediction                                         1.000000
dim_2__mean_change                                 0.503908
dim_4__mean_change                                 0.493050
dim_3__mean_change                                 0.439238
dim_4__time_reversal_asymmetry_

In [ ]:
# To sort features

import re
def extract_fifth_number(column_name):
    match = re.match(r'^.{4}(\d+)', column_name)  # Match numbers after the first 4 characters
    if match:
        return int(match.group(1))
    return float('inf')  # Use infinity for columns without a valid number

# Create a dictionary with column names as keys and extracted numbers as values
column_sort_order = {col: extract_fifth_number(col) for col in features.columns}

# Sort columns based on the extracted numbers
sorted_columns = sorted(column_sort_order, key=column_sort_order.get)

# Reorder the DataFrame columns
features_sorted = features[sorted_columns]
features_sorted

,dim_1__variance_larger_than_standard_deviation,dim_1__has_duplicate_max,dim_1__has_duplicate_min,dim_1__has_duplicate,dim_1__sum_values,dim_1__abs_energy,dim_1__mean_abs_change,dim_1__mean_change,dim_1__mean_second_derivative_central,dim_1__median,...,dim_24__fourier_entropy__bins_10,dim_24__fourier_entropy__bins_100,dim_24__permutation_entropy__dimension_3__tau_1,dim_24__permutation_entropy__dimension_4__tau_1,dim_24__permutation_entropy__dimension_5__tau_1,dim_24__permutation_entropy__dimension_6__tau_1,dim_24__permutation_entropy__dimension_7__tau_1,dim_24__query_similarity_count__query_None__threshold_0.0,dim_24__mean_n_absolute_max__number_of_maxima_7,target
0,1.0,0.0,0.0,0.0,0.030129,144.171555,0.053482,0.003926,-0.000020,-0.181655,...,0.197570,0.359959,1.356889,2.017574,2.645180,3.193317,3.690775,0.0,1.893476,19.38
1,0.0,0.0,0.0,0.0,0.070039,143.752557,0.060102,-0.007028,-0.000086,-0.269548,...,0.144565,0.393377,1.490605,2.169506,2.704489,3.199043,3.635333,0.0,1.735224,12.68
2,0.0,0.0,0.0,0.0,-0.055389,143.549553,0.049657,-0.017667,0.000615,-0.281585,...,0.144565,0.359959,1.656047,2.615860,3.397440,3.882123,4.220529,0.0,2.020656,5.34
3,0.0,0.0,0.0,0.0,-0.102545,143.626298,0.065301,0.018032,-0.000291,-0.169591,...,0.216560,0.216560,1.430972,2.147072,2.736767,3.182333,3.545052,0.0,1.684670,12.72
4,1.0,0.0,0.0,0.0,-0.024506,144.195081,0.049881,-0.007979,-0.000085,-0.155485,...,0.216560,0.359959,1.518376,2.202989,2.730881,3.127549,3.446732,0.0,2.295439,13.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,0.0,0.0,0.0,0.0,0.197303,143.692474,0.082391,0.001954,0.001743,-0.077312,...,0.197570,0.216560,1.438536,2.202767,2.828864,3.248842,3.540016,0.0,1.961003,14.71
91,0.0,0.0,0.0,0.0,-0.048036,143.830875,0.056887,0.007741,-0.000431,-0.107052,...,0.288359,0.483562,1.401238,2.105678,2.718789,3.247124,3.683607,0.0,1.873215,13.69
92,1.0,0.0,0.0,0.0,0.074534,144.055532,0.045370,-0.009928,0.000825,-0.189990,...,0.144565,0.457404,1.312840,1.986927,2.580288,3.011074,3.379185,0.0,2.316813,13.87
93,0.0,0.0,0.0,0.0,-0.068987,143.908933,0.069063,0.012539,-0.000115,-0.014112,...,0.359959,0.573539,1.202433,1.658048,2.066516,2.433053,2.790401,0.0,2.256027,17.66


In [ ]:
file_path = '/home/sim/Desktop/TS Extrinsic Regression/0.5_dct_AppliancesEnergy_20min_Flaml_f/selected_features.txt'
with open(file_path, 'r') as file:
    lines = file.readlines()

lines = [line.strip() for line in lines]

selected_features = features[lines]


column_sort_order = {col: extract_fifth_number(col) for col in selected_features.columns}

# Sort columns based on the extracted numbers
sorted_columns = sorted(column_sort_order, key=column_sort_order.get)

# Reorder the DataFrame columns
sfeatures_sorted = selected_features[sorted_columns]
sfeatures_sorted

,dim_1__fft_coefficient__attr_real__coeff_54,dim_1__agg_linear_trend__attr_slope__chunk_len_50__f_agg_var,dim_1__fft_coefficient__attr_angle__coeff_55,dim_1__agg_linear_trend__attr_rvalue__chunk_len_50__f_agg_max,dim_1__fft_coefficient__attr_real__coeff_10,dim_1__fft_coefficient__attr_imag__coeff_30,dim_1__friedrich_coefficients__coeff_2__m_3__r_30,dim_1__fft_coefficient__attr_angle__coeff_18,dim_1__fft_coefficient__attr_angle__coeff_11,dim_1__ar_coefficient__coeff_4__k_10,...,dim_24__fft_coefficient__attr_angle__coeff_9,dim_24__autocorrelation__lag_2,dim_24__change_quantiles__f_agg_var__isabs_True__qh_1.0__ql_0.8,dim_24__kurtosis,dim_24__change_quantiles__f_agg_var__isabs_False__qh_1.0__ql_0.6,dim_24__fft_coefficient__attr_real__coeff_9,dim_24__cwt_coefficients__coeff_13__w_2__widths_(2 5 10 20),dim_24__abs_energy,dim_24__fft_coefficient__attr_abs__coeff_66,dim_24__agg_linear_trend__attr_intercept__chunk_len_10__f_agg_var
0,0.156626,0.600406,100.892866,0.247173,-1.260077,0.478149,-0.001483,111.175615,65.082276,-0.168497,...,5.290219,0.981040,0.001089,-0.834957,0.007414,0.925968,0.204903,143.972744,0.269521,0.009670
1,0.513801,-0.055231,-99.392647,-0.997455,2.197962,-0.638151,0.002013,-78.162471,-12.287728,-0.071866,...,-140.879038,1.000255,0.002885,-1.403646,0.004978,-0.725611,0.034512,144.128178,0.143881,0.022109
2,0.940263,-0.152099,-26.578071,-0.964726,2.192180,-1.352877,-0.019996,-40.104958,-67.892272,-0.361863,...,59.635098,0.979068,0.004171,-0.624916,0.005247,1.591491,-0.347373,144.392614,1.059552,0.024537
3,-1.653940,0.478783,154.010074,0.454243,-2.955888,1.789823,0.012876,114.000213,109.649353,-0.007616,...,90.808337,0.991840,0.000562,-1.512424,0.004309,-0.047917,-0.007490,143.813904,0.426118,0.064666
4,0.160933,0.064371,-6.429819,-0.948230,0.515984,-0.553009,-0.007138,-12.974975,-75.019189,0.124707,...,106.529598,0.967575,0.009351,-0.427642,0.008709,-2.162029,0.166171,142.408118,0.452589,0.004455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,0.152326,0.276977,-53.278456,-0.335404,-0.333584,0.291328,0.018697,90.472273,81.779086,0.129179,...,113.247055,0.984654,0.001155,-0.536596,0.001851,-2.469247,0.029226,144.023734,0.897189,0.004539
91,0.088679,0.398727,-158.022950,0.514211,0.674679,0.508203,0.030849,175.267841,100.803033,0.040584,...,-45.868069,0.976471,0.001539,-1.287492,0.007731,5.418900,0.009268,143.685469,0.799995,0.036925
92,0.680078,-0.038929,-8.848225,-0.993408,1.433529,-1.077931,0.007233,-39.281501,-72.937957,-0.125048,...,-152.600924,0.963897,0.013362,-0.218809,0.025028,-5.041996,-0.036159,143.871190,0.321591,-0.008010
93,-0.776863,0.014252,136.768366,-0.862088,-0.173653,0.377408,0.006801,119.648463,62.724579,-0.153189,...,122.257420,0.978218,0.002839,0.355987,0.017445,-2.893243,-0.156202,144.061625,0.620969,0.026779
